<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#스키마" data-toc-modified-id="스키마-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>스키마</a></span></li><li><span><a href="#칼럼과-표현식" data-toc-modified-id="칼럼과-표현식-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>칼럼과 표현식</a></span><ul class="toc-item"><li><span><a href="#칼럼" data-toc-modified-id="칼럼-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>칼럼</a></span></li><li><span><a href="#표현식" data-toc-modified-id="표현식-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>표현식</a></span></li><li><span><a href="#레코드와-로우" data-toc-modified-id="레코드와-로우-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>레코드와 로우</a></span></li></ul></li></ul></div>

In [1]:
# 데이터 로드
df = spark.read.format('json').load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/flight-data/json/2015-summary.json')

In [2]:
# 스키마 살펴보기
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



# 스키마

In [4]:
spark.read.format('json').load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/flight-data/json/2015-summary.json').schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

- 스키마를 출력한 결과
    * ``StructType``이라는 리스트 안에 각 칼럼별로 ``StructField``가 담겨져 있음
    * 즉, 스키마 = 여러개의 ``StructField`` 타입 필드로 구성된 ``StructType`` 객체
- 데이터 타입과 스키마의 데이터 타입이 불일치하면 스파크에서는 오류 발생
- 스파크에서는 자체 데이터 타입을 사용하므로 사용하는 언어 API(ex.Python, R 등)의 데이터 타입을 사용할 수 없음

- 스키마 정의시 **메타 데이터**를 정의할 수도 있음
    * **메타 데이터란, 해당 칼럼과 관련된 정보이며 추후 스파크의 머신러닝 라이브러리에서 사용함**

In [5]:
# 데이터프레임에 스키마를 만들고 적용하는 예제
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField('DEST_COUNTRY_NAME', StringType(), True),
    StructField('ORIGIN_COUNTRY_NAME', StringType(), True),
    StructField('count', LongType(), False, metadata={'hello': 'world'})
])

# 데이터 로드 시 스키마 정의
df = spark.read.format('json').schema(myManualSchema)\
     .load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/flight-data/json/2015-summary.json')

# 칼럼과 표현식
- 표현식으로 데이터프레임의 칼럼을 선택, 조작, 제거 가능
- 스파크의 칼럼은 표현식을 사용해 레코드(Row) 단위로 계산한 값을 단순하게 나타내는 논리적인 구조
- 칼럼의 실제값을 얻으려면 레코드(Row)가 필요하고 레코드를 얻으려면 데이터프레임이 필요

## 칼럼

- 칼럼을 생성, 참조할 수 있는 방법은 여러가지가 있지만 ``col()``이나 ``column()``을 사용하는 것이 가장 간단. 소괄호 인자에는 ``칼럼명``을 입력

In [6]:
from pyspark.sql.functions import col, column

col('someColumnName')
column('someColumnName')

Column<b'someColumnName'>

- 이 때 해당 칼럼이 로드한 DataFrame에 있는지 여부는 알 수 없음. 알려면 **카탈로그**에 저장된 정보와 비교해야 함. 그런데 이 비교하는 단계는 **분석기**가 트랜스포메이션으로 구성된 논리적실행계획을 검증 전/후로 가는 단계에 걸쳐있음

- 명시적 컬럼 참조. ``col()`` 사용. 이는 데이터프레임 조인 시 특정 칼럼을 참조하는 데 사용
- 단, ``col()``을 사용해 명시적으로 컬럼을 정의하면 **분석기** 실행 단계에서 컬럼 확인 절차를 생략함!(그러므로 더 빨라지겠지..?)
- Pyspark에서는 ``df['column_name']``을 사용

In [10]:
df['count']

Column<b'count'>

## 표현식
- 표현식이란, 데이터프레임의 레코드의 여러값에 대한 **트랜스포메이션의 집합**
- ``expr()``과 ``col()``로 특정 칼럼을 참조하는 것은 동일
- 예를 들어, ``expr('SomeCol - 5')`` == ``col('someCol') - 5`` == ``expr(SomeCol') - 5``

In [11]:
from pyspark.sql.functions import expr

expr("(((SomeCol + 5) * 200) - 6) < otherCol")

Column<b'((((SomeCol + 5) * 200) - 6) < otherCol)'>

- 위 데이터프레임 코드는 SQL의 SELECT 구문으로 해당 표현식을 동일하게 사용해도 동일한 결과를 생성. 왜냐하면 실행 시점에 동일한 논리 트리로 컴파일되기 때문. 성능도 동일

In [12]:
# 데이터프레임의 컬럼에 접근해보기
spark.read.format('json').load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/flight-data/json/2015-summary.json')\
     .columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

## 레코드와 로우

In [13]:
# 가장 첫 번째 레코드(Row 객체)를 반환
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

- Row객체인 레코드를 생성해보자
- 단, Row는 스키마를 갖고 있지 않고 데이터프레임만 스키마를 가짐. 따라서 Row 생성 시, 데이터프레임의 명시된 스키마 순서와 동일하게 해주어야 함

In [14]:
from pyspark.sql import Row

myRow = Row('Hello', None, 1, False) # 마지막 False는 nullable에 대한 인자!?

In [20]:
# 로우의 데이터에 접근하기
print(myRow[0], myRow[1])

Hello None


In [17]:
Row?